<a href="https://colab.research.google.com/github/elangbijak4/Adaptive-Stratified-Sampling---ASS/blob/main/ASS_Versi_Pilih_dengan_Adil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

# --- 1. SIMULASI POPULASI ---
# Mensimulasikan data: Rata-rata sama, tetapi keragaman (Varians) berbeda
np.random.seed(10)
POP_SIZE = 5000
strata_data = {
    'Strata_1': np.random.normal(loc=50, scale=5, size=POP_SIZE),  # Varians Rendah (Homogen)
    'Strata_2': np.random.normal(loc=50, scale=15, size=POP_SIZE), # Varians Sedang
    'Strata_3': np.random.normal(loc=50, scale=30, size=POP_SIZE) # Varians TINGGI (Paling Buruk)
}
populasi = pd.DataFrame(strata_data)

print("--- ASS Berbasis Efisiensi (Keadilan/Varians) ---")
print(f"Rata-rata Populasi Sejati Total: {populasi.values.mean():.2f}")
print("-" * 40)

# --- 2. PENAMBILAN SAMPEL AWAL DAN PERHITUNGAN VARIAN Awal ---
N_initial = 10
initial_samples = {}
initial_stds = {} # Simpangan Baku (ukuran keragaman/ketidakpastian)

print(f"Mengambil sampel awal N={N_initial} dan menghitung ketidakpastian (Simpangan Baku)...")
for strata, data in strata_data.items():
    sample = np.random.choice(data, size=N_initial, replace=False)
    initial_samples[strata] = sample
    std_dev = sample.std()
    initial_stds[strata] = std_dev
    print(f"  {strata}: Simpangan Baku Awal (Sh) = {std_dev:.2f}")

# --- 3. KALIBRASI (PRINSIP KEADILAN/NEYMAN) ---
Total_M_Budget = 150 # Total sampel tambahan yang dialokasikan
# Alokasi M ~ Sh: Semakin tinggi Simpangan Baku (keragaman), semakin banyak sampel tambahan dialokasikan.
total_std = sum(initial_stds.values())

additional_allocation = {}
print(f"\nMelakukan Kalibrasi (Alokasi Keadilan) dengan Total Budget M={Total_M_Budget}...")

for strata, std in initial_stds.items():
    # Hitung proporsi Sh terhadap total Sh, lalu kalikan dengan budget M
    proporsi = std / total_std
    M_strata = round(proporsi * Total_M_Budget)
    additional_allocation[strata] = M_strata

    # Filosofi Keadilan/Efisiensi:
    if std > initial_stds['Strata_1']: # Jika lebih bervariasi (lebih buruk)
        action = "TAMBAH LEBIH BANYAK"
    else: # Jika kurang bervariasi (lebih baik)
        action = "TAMBAH SEDIKIT (Efisien)"

    print(f"  {strata} (Sh={std:.2f}): Proporsi={proporsi:.2f}. Alokasi M: {M_strata} ({action})")

# --- 4. ADAPTASI DAN PERHITUNGAN AKHIR ---
final_means = {}

print("\nMelakukan Adaptasi (Pengambilan Sampel Tambahan)...")
for strata, M_add in additional_allocation.items():

    # Total ukuran sampel akhir
    final_size = N_initial + M_add

    # Mengambil sampel tambahan
    data = strata_data[strata]
    remaining_data = [x for x in data if x not in initial_samples[strata]]
    additional_sample = np.random.choice(remaining_data, size=M_add, replace=False)
    final_sample = np.concatenate([initial_samples[strata], additional_sample])

    final_means[strata] = final_sample.mean()
    print(f"  {strata}: Sampel Akhir = {final_size}. Rata-rata Akhir: {final_means[strata]:.2f}")

# --- 5. ESTIMASI AKHIR (Membandingkan Distribusi Sampel) ---
# Total estimasi rata-rata menggunakan bobot N_h/N_populasi
true_mean = populasi.values.mean()

# Estimasi Bobot Strata (W_h)
W_strata = 1/3

# Estimasi Akhir Berbasis Adaptasi
est_with_adapt = sum(final_means.values()) * W_strata

print("\n" + "=" * 60)
print(f"Rata-rata Populasi Sejati Total: {true_mean:.2f}")
print("-" * 60)
print(f"Estimasi DENGAN Adaptasi (N + Alokasi Keadilan): {est_with_adapt:.2f}")
print("Hasil: Alokasi DENGAN Keadilan (proporsional Varians) memastikan strata yang paling sulit")
print("diestimasi (varians tinggi) mendapat sampel terbanyak, menghasilkan estimasi yang lebih presisi.")
print("=" * 60)

--- ASS Berbasis Efisiensi (Keadilan/Varians) ---
Rata-rata Populasi Sejati Total: 50.10
----------------------------------------
Mengambil sampel awal N=10 dan menghitung ketidakpastian (Simpangan Baku)...
  Strata_1: Simpangan Baku Awal (Sh) = 4.94
  Strata_2: Simpangan Baku Awal (Sh) = 16.79
  Strata_3: Simpangan Baku Awal (Sh) = 23.19

Melakukan Kalibrasi (Alokasi Keadilan) dengan Total Budget M=150...
  Strata_1 (Sh=4.94): Proporsi=0.11. Alokasi M: 17 (TAMBAH SEDIKIT (Efisien))
  Strata_2 (Sh=16.79): Proporsi=0.37. Alokasi M: 56 (TAMBAH LEBIH BANYAK)
  Strata_3 (Sh=23.19): Proporsi=0.52. Alokasi M: 77 (TAMBAH LEBIH BANYAK)

Melakukan Adaptasi (Pengambilan Sampel Tambahan)...
  Strata_1: Sampel Akhir = 27. Rata-rata Akhir: 50.56
  Strata_2: Sampel Akhir = 66. Rata-rata Akhir: 50.82
  Strata_3: Sampel Akhir = 87. Rata-rata Akhir: 48.95

Rata-rata Populasi Sejati Total: 50.10
------------------------------------------------------------
Estimasi DENGAN Adaptasi (N + Alokasi Keadilan):